In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [4]:
# Paths to dataset directories
train_data_dir = '/kaggle/input/stanford-cars-dataset/cars_train/cars_train'
test_data_dir = '/kaggle/input/stanford-cars-dataset/cars_test/cars_test'
train_csv_path = '/kaggle/input/stanford-car-csv-file/cardatasettrain.csv'
test_csv_path = '/kaggle/input/stanford-car-csv-file/cardatasettest.csv'

In [5]:
# Load the CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Extract image paths and labels from the dataframes
train_images = train_df['image'].values
train_labels = train_df['Class'].values

# Adjust labels to start from 0
train_labels = train_labels - 1

# Convert lists to numpy arrays
train_images = np.array([os.path.join(train_data_dir, img) for img in train_images])
train_labels = np.array(train_labels)

# Print unique labels to check for any out of range values
print(f"Unique labels in the dataset: {np.unique(train_labels)}")

# Ensure all labels are within the valid range
valid_labels = train_labels < 196
train_images = train_images[valid_labels]
train_labels = train_labels[valid_labels]

Unique labels in the dataset: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195]


In [6]:
# First split into train and temp sets
train_images, temp_images, train_labels, temp_labels = train_test_split(train_images, train_labels, test_size=0.4, random_state=42)

# Further split temp set into validation and test sets
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)

# Define the actual number of categories
num_car_types = 196  # Number of car types

In [7]:
print(f'train_images length= {len(train_images)}  test_images length= {len(test_images)}  val_images length= {len(val_images)}')

train_images length= 4886  test_images length= 1629  val_images length= 1629


In [8]:
# Function to load and preprocess images
def load_image(img_path, target_size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array /= 255.
    return img_array

# Data generator for training and validation images
def data_generator(img_paths, labels, batch_size=32):
    while True:
        for start in range(0, len(img_paths), batch_size):
            end = min(start + batch_size, len(img_paths))
            batch_images = []
            batch_labels = []
            for img_path, label in zip(img_paths[start:end], labels[start:end]):
                if label >= num_car_types:
                    print(f"Label {label} is out of range for {num_car_types} classes.")
                img_array = load_image(img_path)
                batch_images.append(img_array)
                batch_labels.append(label)
            yield np.array(batch_images), tf.keras.utils.to_categorical(batch_labels, num_classes=num_car_types)

In [9]:
from tensorflow.keras.layers import Dropout, BatchNormalization

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

# Output layer for car type
output_car_type = Dense(num_car_types, activation='softmax', name='car_type')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output_car_type)

# Unfreeze some layers in the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [ tf.keras.callbacks.EarlyStopping( monitor="val_loss", min_delta=0,  patience=20, verbose=1, mode="auto", baseline=None, restore_best_weights=True)]

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,890,884 (98.77 MB)

 Trainable params: 25,835,716 (98.56 MB)

 Non-trainable params: 55,168 (215.50 KB)

In [10]:
# Train the model
batch_size = 32
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(val_images) // batch_size

history_model = model.fit(
    data_generator(train_images, train_labels, batch_size=batch_size),
    epochs=50,
    callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_data=data_generator(val_images, val_labels, batch_size=batch_size),
    validation_steps=validation_steps
)

Epoch 1/50


I0000 00:00:1719729501.685220     231 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719729501.778334     231 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.0037 - loss: 6.0490

W0000 00:00:1719729546.440197     229 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 ━━━━━━━━━━━━━━━━━━━━ 130s 354ms/step - accuracy: 0.0038 - loss: 6.0488 - val_accuracy: 0.0094 - val_loss: 5.5830
Epoch 2/50


W0000 00:00:1719729594.990856     230 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accuracy: 0.0224 - loss: 5.4216

W0000 00:00:1719729646.992237     230 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 ━━━━━━━━━━━━━━━━━━━━ 99s 392ms/step - accuracy: 0.0223 - loss: 5.4218 - val_accuracy: 0.0094 - val_loss: 6.0019
Epoch 3/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 50s 329ms/step - accuracy: 0.0462 - loss: 4.9371 - val_accuracy: 0.0081 - val_loss: 6.3309
Epoch 4/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 49s 326ms/step - accuracy: 0.0899 - loss: 4.4802 - val_accuracy: 0.0106 - val_loss: 6.5838
Epoch 5/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 48s 318ms/step - accuracy: 0.1477 - loss: 4.1053 - val_accuracy: 0.0081 - val_loss: 6.1673
Epoch 6/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 47s 313ms/step - accuracy: 0.2329 - loss: 3.6847 - val_accuracy: 0.0207 - val_loss: 5.5017
Epoch 7/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 48s 314ms/step - accuracy: 0.3276 - loss: 3.3327 - val_accuracy: 0.0319 - val_loss: 5.0862
Epoch 8/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 48s 314ms/step - accuracy: 0.4231 - loss: 2.9935 - val_accuracy: 0.0520 - val_loss: 4.9500
Epoch 9/50
152/152 ━━━━━━━━━━━━━━━━━━━━ 48s 313ms/step - accuracy: 0.5449 - loss: 2.6701 - val

In [11]:
# Evaluate the model
test_steps = len(test_images) // batch_size
test_generator = data_generator(test_images, test_labels, batch_size=batch_size)
test_loss, test_acc = model.evaluate(test_generator, steps=test_steps)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 403ms/step - accuracy: 0.1430 - loss: 4.3222
Test Loss: 4.3771562576293945
Test Accuracy: 0.13249999284744263
